# YOLOv12人脸关键点检测模型测试

这个笔记本用于测试YOLOv12人脸关键点检测模型的训练和评估。

## 1. 环境设置

首先克隆YOLOv12仓库并安装依赖

In [ ]:
# 克隆仓库
!git clone https://github.com/YourRepo/yolov12.git
!cd yolov12 && pip install -e .

## 2. 数据集准备

假设我们已经有了处理好的人脸关键点数据集，这里我们将创建一个简单的示例数据

In [ ]:
import os
import yaml
import shutil
from pathlib import Path

# 创建数据目录结构
data_dir = Path('face_data')
os.makedirs(data_dir / 'train' / 'images', exist_ok=True)
os.makedirs(data_dir / 'train' / 'labels', exist_ok=True)
os.makedirs(data_dir / 'val' / 'images', exist_ok=True)
os.makedirs(data_dir / 'val' / 'labels', exist_ok=True)

# 创建数据集配置文件
data_yaml = {
    'path': str(data_dir.absolute()),
    'train': 'train',
    'val': 'val',
    'nc': 1,
    'names': ['face'],
    'kpt_shape': [5, 3],  # 5个关键点，每个点有3个值 [x,y,visible]
    'flip_idx': [1, 0, 2, 4, 3],  # 水平翻转时的关键点对应关系
    'kpt_names': ['left_eye', 'right_eye', 'nose', 'left_mouth', 'right_mouth']
}

with open(data_dir / 'data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"数据集配置已创建: {data_dir / 'data.yaml'}")

## 3. 模型训练

使用YOLOv12的人脸关键点检测模型进行训练

In [ ]:
from ultralytics import YOLO

# 创建模型 - 使用yolov12-face配置
model = YOLO('yolov12n-face.yaml')

# 开始训练
results = model.train(
    data=str(data_dir / 'data.yaml'),  # 数据集配置路径
    epochs=100,                        # 训练轮数
    imgsz=640,                         # 图像大小
    batch=16,                          # 批次大小
    patience=20,                       # 早停耐心值
    save=True,                         # 保存模型
    project='face_runs',               # 项目名称
    name='train_face',                 # 运行名称
    device=0                           # 使用GPU
)

## 4. 模型验证

验证训练完成的模型性能

In [ ]:
# 加载最佳模型
best_model = YOLO('face_runs/train_face/weights/best.pt')

# 在验证集上验证模型
val_results = best_model.val(
    data=str(data_dir / 'data.yaml'),  # 数据集配置路径
    imgsz=640,                         # 图像大小
    batch=16,                          # 批次大小
    plots=True                         # 生成评估图表
)

# 打印验证结果
print(f"边界框mAP50: {val_results.box.map50:.4f}")
print(f"边界框mAP50-95: {val_results.box.map:.4f}")
if hasattr(val_results, 'pose'):
    print(f"关键点mAP50: {val_results.pose.map50:.4f}")
    print(f"关键点mAP50-95: {val_results.pose.map:.4f}")

## 5. 模型预测

使用训练好的模型进行人脸关键点检测预测

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# 加载图像进行预测（使用示例图像）
# 实际应用中，替换为你自己的图像路径
image_path = 'path/to/test/image.jpg'

# 运行预测
results = best_model.predict(
    source=image_path,
    conf=0.25,                 # 置信度阈值
    save=True,                 # 保存结果
    project='face_runs',       # 项目名称
    name='predict_face',       # 运行名称
    show_labels=True,          # 显示标签
    show_conf=True,            # 显示置信度
    line_width=2               # 边界框线宽
)

# 显示结果
for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    plt.figure(figsize=(12, 8))
    plt.imshow(im)
    plt.axis('off')
    plt.show()

## 6. 模型导出

将训练好的模型导出为不同格式，用于部署

In [ ]:
# 将模型导出为ONNX格式
best_model.export(format='onnx')

# 可选：导出为其他格式
# 可用格式包括：'torchscript'，'onnx'，'openvino'，'engine'，'coreml'，'saved_model'，'pb'，'tflite'，'edgetpu'，'tfjs'，'paddle'
# best_model.export(format='engine', half=True)  # 导出为TensorRT引擎，使用半精度

## 总结

我们已经完成了YOLOv12人脸关键点检测模型的完整工作流程，包括：

1. 环境设置
2. 数据集准备
3. 模型训练
4. 模型验证
5. 模型预测
6. 模型导出

这个流程可以用于实际项目中的人脸关键点检测任务。